In [ ]:
import time
import threading
import requests
import re
import pandas as pd
from tqdm import tqdm
import pymysql
from multiprocessing.dummy import Pool

def request_(page_number):
    """
    请求网页
    """
    global error_url
    global res
    # lock.acquire()    #线程锁
    addr = "0x2d50c01669949df3bd21142f8f20ab14c48d1567"
    key = "ckey_2545e4a689bb4d4c87d64889755"
    page_size = 10
    
    while True:
        params = {
            "page-number":f"{page_number}",
            "page-size":f"{page_size}",
            "key":key
            }
        url = f"https://api.covalenthq.com/v1/56/address/{addr}/transactions_v2/?quote-currency=USD&page-size={page_size}&page-number={page_number}"
        try:
            r = requests.get(url,params=params)
        except Exception as e:
            print(f"捕获异常{e}")
            print("捕获异常，等待 {} 秒后继续尝试请求！".format(3))
            time.sleep(3)
            continue

        if (r.status_code == 200) and (len(r.json()['data']['items'])>0):
            for i in r.json()['data']['items']:
                try:
                    if (i['successful'] == True) and (0 < len(i['log_events']) <= 2) and (i["to_address_label"]==None):
                        date = i['block_signed_at'].replace('T',' ').replace('Z','') #交易日期
                        tx_hash = i['tx_hash'] #交易哈希
                        from_address = i['from_address'] #交易来源
                        raw_log_data = i['log_events'][-1]['raw_log_data'] #交易日志
                        to_address = '0x'+raw_log_data[-40:] #交易去处
                        
                        res['date'].append(date)
                        res['tx_hash'].append(tx_hash)
                        res['from_address'].append(from_address)
                        res['to_address'].append(to_address)
                        
                except Exception as e:
                    print(e)
                    error_url.append([e,i])
                    continue
                    
        elif (r.status_code == 200) and (len(r.json()['data']['items'])==0):
            # lock.release()    #释放锁
            pool_sema.release() #释放已完成线程
            return pd.DataFrame(res)
        
        elif r.status_code != 200: 
            time.sleep(1)
            continue
            
            
            
error_url = []
res = {'date':[],'tx_hash':[],'from_address':[],'to_address':[]}

lock = threading.Lock() #设置线程锁
max_connection = 4 #4线程
pool_sema = threading.BoundedSemaphore(max_connection) #最大线程量

thread_list = []
for i in tqdm(range(1,10)):
    pool_sema.acquire() 
    thread = threading.Thread(target=request_,args=[i])
    thread.start()
    thread_list.append(thread)

for t in thread_list:
    t.join()
    

In [9]:
from scrapy.http import HtmlResponse
r = HtmlResponse(url='')

In [17]:
not r.url

True

In [1]:
import requests

In [28]:
r = requests.get('https://api.covalenthq.com/v1/56/address/0xAaA84DeE0C325d85B16DCfBa7d31390888B7e512/transactions_v2/?quote-currency=USD&page-size=10&page-number=1&key=ckey_2545e4a689bb4d4c87d64889755')

In [29]:
r.status_code

200

In [32]:
r.json()

[{'block_signed_at': '2022-08-21T05:36:52Z',
  'block_height': 20627110,
  'tx_hash': '0xe397ca9e322f8e79b44a37b4bf1e1af7e32a3717d30182f0bd97cf5e850c88c8',
  'tx_offset': 41,
  'successful': True,
  'from_address': '0xc252bc7fa845bd115f1bcdc18dc90a067e201450',
  'from_address_label': None,
  'to_address': '0xaaa84dee0c325d85b16dcfba7d31390888b7e512',
  'to_address_label': None,
  'value': '0',
  'value_quote': None,
  'gas_offered': 263980,
  'gas_spent': 175987,
  'gas_price': 5000000000,
  'fees_paid': '879935000000000',
  'gas_quote': None,
  'gas_quote_rate': None,
  'log_events': [{'block_signed_at': '2022-08-21T05:36:52Z',
    'block_height': 20627110,
    'tx_offset': 41,
    'log_offset': 73,
    'tx_hash': '0xe397ca9e322f8e79b44a37b4bf1e1af7e32a3717d30182f0bd97cf5e850c88c8',
    'raw_log_topics': ['0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925',
     '0x000000000000000000000000c252bc7fa845bd115f1bcdc18dc90a067e201450',
     '0x000000000000000000000000aaa84